# Дополнительное задание к лабораторной №1: 
---
На основании биграмм, сгенерировать текст. 
Взять самую высокую частоту биграммы, и использовать ее в качестве "затравки", пока не наберется как минимум 30 слов
---

In [1]:
import nltk
from nltk import ngrams
from nltk.tokenize import RegexpTokenizer

import re
import pandas as pd
from typing import Dict, Union, Optional, Iterable, List, Match, Tuple, Callable


nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\79992\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

---
### Использованные книги:
---
1. [GIVE THE BOYS A GREAT BIG HAND](https://www.rulit.me/books/give-the-boys-a-great-big-hand-download-679500.html)
2. [Treasure Island by Robert Louis Stevenson](https://www.gutenberg.org/ebooks/120) 
3. [Great Expectations by Charles Dickens](https://www.gutenberg.org/ebooks/1400) 
4. [Hamlet, Prince of Denmark by William Shakespeare](https://www.gutenberg.org/ebooks/1524) 
5. [TUBE TO NOWHERE](https://www.rulit.me/books/tube-to-nowhere-download-679490.html)
6. [YOU HAVE YOURSELF A DEAL](https://www.rulit.me/books/you-have-yourself-a-deal-download-679488.html)
7. [THE WAY THE COOKIE CRUMBLES](https://www.rulit.me/books/the-way-the-cookie-crumbles-download-679487.html) 
8. [YOU MUST BE KIDDING](https://www.rulit.me/books/you-must-be-kidding-download-679486.html) 
9. [The Odyssey by Homer](https://www.gutenberg.org/ebooks/1727) 
10. [DOOM ON THE HILL](https://www.rulit.me/books/doom-on-the-hill-download-679371.html)
11. [Simple Sabotage Field Manual by United States. Office of Strategic Services](https://www.gutenberg.org/ebooks/26184)
12. [THE CHINESE DISKS](https://www.rulit.me/books/the-chinese-disks-download-679286.html) 
13. [The Mysterious Affair at Styles by Agatha Christie](https://www.gutenberg.org/ebooks/863) 
14. [Beowulf: An Anglo-Saxon Epic Poem by J. Lesslie Hall](https://www.gutenberg.org/ebooks/16328) 
15. [NEVER FALL FOR YOUR FIANCEE](https://www.rulit.me/books/never-fall-for-your-fiancee-download-678785.html)
16. [Anna Karenina by graf Leo Tolstoy](https://www.gutenberg.org/ebooks/1399)
17. [On Liberty by John Stuart Mill](https://www.gutenberg.org/ebooks/34901) 
18. [THE GARAUCAN SWINDLE](https://www.rulit.me/books/the-garaucan-swindle-download-678697.html) 
19. [SPOILS OF THE SHADOW](https://www.rulit.me/books/spoils-of-the-shadow-download-678653.html) 
20. [Old Granny Fox by Thornton W. Burgess](https://www.gutenberg.org/ebooks/4980)

In [2]:
with open(r'data/books.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
len(text)

11506158

In [3]:
DIGIT_GROUP = re.compile(r"(\d)")
TWO_DIGITS = re.compile(r"(\d)(\d)")
THREE_DIGITS = re.compile(r"(\d)(\d)(\d)")
THREE_DIGITS_WORD = re.compile(r"(\d)(\d)(\d)(?=\D*\Z)")
TWO_DIGITS_WORD = re.compile(r"(\d)(\d)(?=\D*\Z)")
ONE_DIGIT_WORD = re.compile(r"(\d)(?=\D*\Z)")

FOUR_DIGIT_COMMA = re.compile(r"(\d)(\d{3}(?:,|\Z))")
NON_DIGIT = re.compile(r"\D")
WHITESPACES_COMMA = re.compile(r"\s+,")
COMMA_WORD = re.compile(r", (\S+)\s+\Z")
WHITESPACES = re.compile(r"\s+")

_number_args = None
mill_count = 0


# NUMERICAL INFLECTIONS

nth = {
    0: "th",
    1: "st",
    2: "nd",
    3: "rd",
    4: "th",
    5: "th",
    6: "th",
    7: "th",
    8: "th",
    9: "th",
    11: "th",
    12: "th",
    13: "th",
}

ordinal = dict(
    ty="tieth",
    one="first",
    two="second",
    three="third",
    five="fifth",
    eight="eighth",
    nine="ninth",
    twelve="twelfth",
)

ordinal_suff = re.compile(fr"({'|'.join(ordinal)})\Z")


# NUMBERS

unit = ["", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
teen = [
    "ten",
    "eleven",
    "twelve",
    "thirteen",
    "fourteen",
    "fifteen",
    "sixteen",
    "seventeen",
    "eighteen",
    "nineteen",
]
ten = [
    "",
    "",
    "twenty",
    "thirty",
    "forty",
    "fifty",
    "sixty",
    "seventy",
    "eighty",
    "ninety",
]
mill = [
    " ",
    " thousand",
    " million",
    " billion",
    " trillion",
    " quadrillion",
    " quintillion",
    " sextillion",
    " septillion",
    " octillion",
    " nonillion",
    " decillion",
]


def millfn(ind: int = 0) -> str:
    if ind > len(mill) - 1:
        print3("number out of range")
        raise NumOutOfRangeError
    return mill[ind]

def unitfn(units: int, mindex: int = 0) -> str:
    return f"{unit[units]}{millfn(mindex)}"

def tenfn(tens, units, mindex=0):
    if tens != 1:
        tens_part = ten[tens]
        if tens and units:
            hyphen = "-"
        else:
            hyphen = ""
        unit_part = unit[units]
        mill_part = millfn(mindex)
        return f"{tens_part}{hyphen}{unit_part}{mill_part}"
    return f"{teen[units]}{mill[mindex]}"

def hundfn(hundreds: int, tens: int, units: int, mindex: int) -> str:
    global _number_args
    
    if hundreds:
        andword = f" {_number_args['andword']} " if tens or units else ""
        # use unit not unitfn as simpler
        return (
            f"{unit[hundreds]} hundred{andword}"
            f"{tenfn(tens, units)}{millfn(mindex)}, "
        )
    if tens or units:
        return f"{tenfn(tens, units)}{millfn(mindex)}, "
    return ""

def group1sub(mo: Match) -> str:
    global _number_args
    
    units = int(mo.group(1))
    if units == 1:
        return f" {_number_args['one']}, "
    elif units:
        return f"{unit[units]}, "
    else:
        return f" {_number_args['zero']}, "

def group1bsub(mo: Match) -> str:
    global _number_args
    units = int(mo.group(1))
    if units:
        return f"{unit[units]}, "
    else:
        return f" {_number_args['zero']}, "

def group2sub(mo: Match) -> str:
    global _number_args
    
    tens = int(mo.group(1))
    units = int(mo.group(2))
    if tens:
        return f"{tenfn(tens, units)}, "
    if units:
        return f" {_number_args['zero']} {unit[units]}, "
    return f" {_number_args['zero']} {_number_args['zero']}, "

def group3sub(mo: Match) -> str:
    global _number_args
    
    hundreds = int(mo.group(1))
    tens = int(mo.group(2))
    units = int(mo.group(3))
    if hundreds == 1:
        hunword = f" {_number_args['one']}"
    elif hundreds:
        hunword = str(unit[hundreds])
    else:
        hunword = f" {_number_args['zero']}"
    if tens:
        tenword = tenfn(tens, units)
    elif units:
        tenword = f" {_number_args['zero']} {unit[units]}"
    else:
        tenword = f" {_number_args['zero']} {_number_args['zero']}"
    return f"{hunword} {tenword}, "

def hundsub(mo: Match) -> str:
    global mill_count
    ret = hundfn(
        int(mo.group(1)), int(mo.group(2)), int(mo.group(3)), mill_count
    )
    mill_count += 1
    return ret

def tensub(mo: Match) -> str:
    global mill_count
    return f"{tenfn(int(mo.group(1)), int(mo.group(2)), mill_count)}, "

def unitsub(mo: Match) -> str:
    global mill_count
    return f"{unitfn(int(mo.group(1)), mill_count)}, "

def enword(num: str, group: int) -> str:
    global _number_args
    global mill_count

    if group == 1:
        num = DIGIT_GROUP.sub(group1sub, num)
    elif group == 2:
        num = TWO_DIGITS.sub(group2sub, num)
        num = DIGIT_GROUP.sub(group1bsub, num, 1)
    elif group == 3:
        num = THREE_DIGITS.sub(group3sub, num)
        num = TWO_DIGITS.sub(group2sub, num, 1)
        num = DIGIT_GROUP.sub(group1sub, num, 1)
    elif int(num) == 0:
        num = _number_args["zero"]
    elif int(num) == 1:
        num = _number_args["one"]
    else:
        num = num.lstrip().lstrip("0")
        mill_count = 0
        # surely there's a better way to do the next bit
        mo = THREE_DIGITS_WORD.search(num)
        while mo:
            num = THREE_DIGITS_WORD.sub(hundsub, num, 1)
            mo = THREE_DIGITS_WORD.search(num)
        num = TWO_DIGITS_WORD.sub(tensub, num, 1)
        num = ONE_DIGIT_WORD.sub(unitsub, num, 1)
    return num

def find_next(bigramm: str, bigramms_table: pd.DataFrame) -> str:
    _, second_word = bigramm.split(' ')
    idx = bigramms_table['N-gramm'].map(lambda x: x.split(' ')[0] == second_word)
    next_bigramm = bigramms_table[idx]
   
    if len(next_bigramm) > 0:
        bigramms_table = bigramms_table.drop(next_bigramm['N-gramm'].index[0])
        return next_bigramm['N-gramm'].iloc[0], bigramms_table
    
    return '<END>', bigramms_table

def replace_nubmers(text):
    def rep_num(nums, text):
        for num in nums:
            text_represenation = number_to_words(num)
            text = text.replace(num, text_represenation)
            print("Numeric: {} replaced by text representation: {}".format(num, text_represenation))
        return text
            
    st = set(re.findall(r'\d+st', text))
    nd = set(re.findall(r'\d+nd', text))
    rd = set(re.findall(r'\d+rd', text))
    th = set(re.findall(r'\d+th', text))
    default_nums = set(re.findall(r'\d+', text))
    
    text = rep_num(st, text)
    text = rep_num(nd, text)
    text = rep_num(rd, text)
    text = rep_num(th, text)
    text = rep_num(default_nums, text)
    
    return text

def token_freq(tokens: List[str]) -> Dict:
    res = {}
    for token in tokens:
        if token in res.keys():
            res[token] += 1
        else:
            res[token] = 1
        
    return res

def freq_df(tokens: List[str]) -> pd.DataFrame:
    ccount = token_freq(tokens)

    col = 'Word'
    word_sample = list(ccount.keys())[0]
    if len(word_sample) > 1:
        col = 'N-gramm'
        
    res = pd.DataFrame({col: ccount.keys(), 'Count': ccount.values()})
    res = res.sort_values('Count', ascending=False).reset_index(drop=True)
    return res

def tokenize_text(text, tokenizer, **kwardgs):
    return tokenizer(text, **kwardgs)

def number_to_words(
        num: Union[int, str],
        wantlist: bool = False,
        group: int = 0,
        comma: str = ",",
        andword: str = "and",
        zero: str = "zero",
        one: str = "one",
        decimal: str = "point",
        threshold: Optional[int] = None,
    ) -> Union[str, List[str]]:
    """
    Return a number in words.

    group = 1, 2 or 3 to group numbers before turning into words
    comma: define comma
    andword: word for 'and'. Can be set to ''.
        e.g. "one hundred and one" vs "one hundred one"
    zero: word for '0'
    one: word for '1'
    decimal: word for decimal point
    threshold: numbers above threshold not turned into words

    parameters not remembered from last call. Departure from Perl version.
    """
    global _number_args
    
    _number_args = {"andword": andword, "zero": zero, "one": one}
    num = str(num)

    # Handle "stylistic" conversions (up to a given threshold)...
    if threshold is not None and float(num) > threshold:
        spnum = num.split(".", 1)
        while comma:
            (spnum[0], n) = FOUR_DIGIT_COMMA.subn(r"\1,\2", spnum[0])
            if n == 0:
                break
        try:
            return f"{spnum[0]}.{spnum[1]}"
        except IndexError:
            return str(spnum[0])

    if group < 0 or group > 3:
        raise BadChunkingOptionError
    nowhite = num.lstrip()
    if nowhite[0] == "+":
        sign = "plus"
    elif nowhite[0] == "-":
        sign = "minus"
    else:
        sign = ""

    myord = num[-2:] in ("st", "nd", "rd", "th")
    if myord:
        num = num[:-2]
    finalpoint = False
    if decimal:
        if group != 0:
            chunks = num.split(".")
        else:
            chunks = num.split(".", 1)
        if chunks[-1] == "":  # remove blank string if nothing after decimal
            chunks = chunks[:-1]
            finalpoint = True  # add 'point' to end of output
    else:
        chunks = [num]

    first: Union[int, str, bool] = 1
    loopstart = 0

    if chunks[0] == "":
        first = 0
        if len(chunks) > 1:
            loopstart = 1

    for i in range(loopstart, len(chunks)):
        chunk = chunks[i]
        # remove all non numeric \D
        chunk = NON_DIGIT.sub("", chunk)
        if chunk == "":
            chunk = "0"

        if group == 0 and (first == 0 or first == ""):
            chunk = enword(chunk, 1)
        else:
            chunk = enword(chunk, group)

        if chunk[-2:] == ", ":
            chunk = chunk[:-2]
        chunk = WHITESPACES_COMMA.sub(",", chunk)

        if group == 0 and first:
            chunk = COMMA_WORD.sub(f" {andword} \\1", chunk)
        chunk = WHITESPACES.sub(" ", chunk)
        
        chunk = chunk.strip()
        if first:
            first = ""
        chunks[i] = chunk

    numchunks = []
    if first != 0:
        numchunks = chunks[0].split(f"{comma} ")

    if myord and numchunks:
        mo = ordinal_suff.search(numchunks[-1])
        if mo:
            numchunks[-1] = ordinal_suff.sub(ordinal[mo.group(1)], numchunks[-1])
        else:
            numchunks[-1] += "th"

    for chunk in chunks[1:]:
        numchunks.append(decimal)
        numchunks.extend(chunk.split(f"{comma} "))

    if finalpoint:
        numchunks.append(decimal)

    if wantlist:
        if sign:
            numchunks = [sign] + numchunks
        return numchunks
    elif group:
        signout = f"{sign} " if sign else ""
        return f"{signout}{', '.join(numchunks)}"
    else:
        signout = f"{sign} " if sign else ""
        num = f"{signout}{numchunks.pop(0)}"
        if decimal is None:
            first = True
        else:
            first = not num.endswith(decimal)
        for nc in numchunks:
            if nc == decimal:
                num += f" {nc}"
                first = 0
            elif first:
                num += f"{comma} {nc}"
            else:
                num += f" {nc}"
        return num

In [4]:
text = replace_nubmers(text)

Numeric: 1st replaced by text representation: first
Numeric: 21st replaced by text representation: twenty-first
Numeric: 31st replaced by text representation: thirty-first
Numeric: 22nd replaced by text representation: twenty-second
Numeric: 2nd replaced by text representation: second
Numeric: 23rd replaced by text representation: twenty-third
Numeric: 3rd replaced by text representation: third
Numeric: 63rd replaced by text representation: sixty-third
Numeric: 18th replaced by text representation: eighteenth
Numeric: 29th replaced by text representation: twenty-ninth
Numeric: 5th replaced by text representation: fifth
Numeric: 15th replaced by text representation: fifteenth
Numeric: 10th replaced by text representation: tenth
Numeric: 20th replaced by text representation: twentieth
Numeric: 87th replaced by text representation: eighty-seventh
Numeric: 11th replaced by text representation: eleventh
Numeric: 13th replaced by text representation: thirteenth
Numeric: 8th replaced by text 

Numeric: 22 replaced by text representation: twenty-two
Numeric: 2522 replaced by text representation: two thousand, five hundred and twenty-two
Numeric: 361 replaced by text representation: three hundred and sixty-one
Numeric: 183 replaced by text representation: one hundred and eighty-three
Numeric: 249 replaced by text representation: two hundred and forty-nine
Numeric: 1452 replaced by text representation: one thousand, four hundred and fifty-two
Numeric: 85 replaced by text representation: eighty-five
Numeric: 1674 replaced by text representation: one thousand, six hundred and seventy-four
Numeric: 204 replaced by text representation: two hundred and four
Numeric: 1994 replaced by text representation: one thousand, nine hundred and ninety-four
Numeric: 175 replaced by text representation: one hundred and seventy-five
Numeric: 3063 replaced by text representation: three thousand and sixty-three
Numeric: 1996 replaced by text representation: one thousand, nine hundred and ninety-six

Numeric: 2500 replaced by text representation: two thousand, five hundred
Numeric: 457 replaced by text representation: four hundred and fifty-seven
Numeric: 1855 replaced by text representation: one thousand, eight hundred and fifty-five
Numeric: 1470 replaced by text representation: one thousand, four hundred and seventy
Numeric: 1527 replaced by text representation: one thousand, five hundred and twenty-seven
Numeric: 192 replaced by text representation: one hundred and ninety-two
Numeric: 1841 replaced by text representation: one thousand, eight hundred and forty-one
Numeric: 146 replaced by text representation: one hundred and forty-six
Numeric: 169 replaced by text representation: one hundred and sixty-nine
Numeric: 67 replaced by text representation: sixty-seven
Numeric: 1546 replaced by text representation: one thousand, five hundred and forty-six
Numeric: 105 replaced by text representation: one hundred and five
Numeric: 54 replaced by text representation: fifty-four
Numeric: 

Numeric: 387 replaced by text representation: three hundred and eighty-seven
Numeric: 1438 replaced by text representation: one thousand, four hundred and thirty-eight
Numeric: 3114 replaced by text representation: three thousand, one hundred and fourteen
Numeric: 1423 replaced by text representation: one thousand, four hundred and twenty-three
Numeric: 61 replaced by text representation: sixty-one
Numeric: 1839 replaced by text representation: one thousand, eight hundred and thirty-nine
Numeric: 15 replaced by text representation: fifteen
Numeric: 2010 replaced by text representation: two thousand and ten
Numeric: 1873 replaced by text representation: one thousand, eight hundred and seventy-three
Numeric: 1501 replaced by text representation: one thousand, five hundred and one
Numeric: 2873 replaced by text representation: two thousand, eight hundred and seventy-three
Numeric: 68 replaced by text representation: sixty-eight
Numeric: 8112 replaced by text representation: eight thousand

Numeric: 102 replaced by text representation: one hundred and two
Numeric: 201 replaced by text representation: two hundred and one
Numeric: 1557 replaced by text representation: one thousand, five hundred and fifty-seven
Numeric: 1526 replaced by text representation: one thousand, five hundred and twenty-six
Numeric: 6221541 replaced by text representation: six million, two hundred and twenty-one thousand, five hundred and forty-one
Numeric: 555 replaced by text representation: five hundred and fifty-five
Numeric: 711 replaced by text representation: seven hundred and eleven
Numeric: 5 replaced by text representation: five
Numeric: 1511 replaced by text representation: one thousand, five hundred and eleven
Numeric: 2053 replaced by text representation: two thousand and fifty-three
Numeric: 1321 replaced by text representation: one thousand, three hundred and twenty-one
Numeric: 306 replaced by text representation: three hundred and six
Numeric: 1824 replaced by text representation: on

In [5]:
# Только буквосодержащие элементы
pattern = r'[^\W\d_]+'
tokenizer = RegexpTokenizer(pattern)
tokens = tokenize_text(text, tokenizer.tokenize)
print(tokens[:200])

['Ed', 'McBain', 'Give', 'the', 'Boys', 'a', 'Great', 'Big', 'Hand', 'This', 'is', 'for', 'Phyllis', 'and', 'Rick', 'one', 'It', 'was', 'raining', 'It', 'had', 'been', 'raining', 'for', 'three', 'days', 'now', 'an', 'ugly', 'March', 'rain', 'that', 'washed', 'the', 'brilliance', 'of', 'near', 'spring', 'with', 'a', 'monochromatic', 'unrelenting', 'gray', 'The', 'television', 'forecasters', 'had', 'correctly', 'predicted', 'rain', 'for', 'today', 'and', 'estimated', 'that', 'it', 'would', 'rain', 'tomorrow', 'also', 'Beyond', 'that', 'they', 'would', 'not', 'venture', 'an', 'opinion', 'But', 'it', 'seemed', 'to', 'Patrolman', 'Richard', 'Genero', 'that', 'it', 'had', 'been', 'raining', 'forever', 'and', 'that', 'it', 'would', 'continue', 'to', 'rain', 'forever', 'and', 'that', 'eventually', 'he', 'would', 'be', 'washed', 'away', 'into', 'the', 'gutters', 'and', 'then', 'carried', 'into', 'the', 'sewers', 'of', 'Isola', 'and', 'dumped', 'unceremoniously', 'with', 'the', 'other', 'garbage

In [6]:
tokens = tokenize_text(text, nltk.word_tokenize)
tokens = [word for word in tokens if word.isalpha()]
print(tokens[:200])

['Ed', 'McBain', 'Give', 'the', 'Boys', 'a', 'Great', 'Big', 'Hand', 'This', 'is', 'for', 'Phyllis', 'and', 'Rick', 'one', 'It', 'was', 'raining', 'It', 'had', 'been', 'raining', 'for', 'three', 'days', 'now', 'an', 'ugly', 'March', 'rain', 'that', 'washed', 'the', 'brilliance', 'of', 'with', 'a', 'monochromatic', 'unrelenting', 'gray', 'The', 'television', 'forecasters', 'had', 'correctly', 'predicted', 'rain', 'for', 'today', 'and', 'estimated', 'that', 'it', 'would', 'rain', 'tomorrow', 'also', 'Beyond', 'that', 'they', 'would', 'not', 'venture', 'an', 'opinion', 'But', 'it', 'seemed', 'to', 'Patrolman', 'Richard', 'Genero', 'that', 'it', 'had', 'been', 'raining', 'forever', 'and', 'that', 'it', 'would', 'continue', 'to', 'rain', 'forever', 'and', 'that', 'eventually', 'he', 'would', 'be', 'washed', 'away', 'into', 'the', 'gutters', 'and', 'then', 'carried', 'into', 'the', 'sewers', 'of', 'Isola', 'and', 'dumped', 'unceremoniously', 'with', 'the', 'other', 'garbage', 'into', 'either

In [7]:
bigramm = [' '.join(i) for i in ngrams(tokens, 2)]
res = freq_df(bigramm)
res.head()

,N-gramm,Count
0,of the,11495
1,in the,8384
2,to the,6007
3,on the,3985
4,and the,3975


In [8]:
def sentence_generator(start: str, n_gramm: pd.DataFrame, number: int) -> str:
    n_gramm2 = n_gramm.copy()
    sentence = ""
    
    for i in range(number):
        start, n_gramm2 = find_next(start, n_gramm2)
        sentence += (start.split(' ')[1] + ' ')
        
    return sentence

In [9]:
first = res['N-gramm'].iloc[0]
print(sentence_generator(first, res, 100))

same time to the other and the first time I had been a little and I was a man who had a few minutes to be a good and then I have been in the door and he had to me to have to my prick in a very much as I could not to her and a long time and that I am not be the most of the time he was the house and it was not a great and was in his head and we had not the man and she had no more than the Project 


---
**Предложение, составленное из 100 самых популярных биграмм:** <br><br>
`same time to the other and the first time I had been a little and I was a man who had a few minutes to be a good and then I have been in the door and he had to me to have to my prick in a very much as I could not to her and a long time and that I am not be the most of the time he was the house and it was not a great and was in his head and we had not the man and she had no more than the Project`